In [3]:
import json

with open('data/squad2/train-v2.0.json', 'r') as file:
    data = json.load(file)

In [15]:
impossible_count = 0

for item in data['data']:
    for paragraph in item['paragraphs']:
        for qas in paragraph['qas']:
            if qas.get('is_impossible', False): 
                impossible_count += 1

print(f"Number of impossible questions: {impossible_count}")

Number of impossible questions: 43498


In [16]:
total_questions = sum(len(paragraph['qas']) for item in data['data'] for paragraph in item['paragraphs'])

print(f"Total number of questions: {total_questions}")

Total number of questions: 130319


In [20]:
impossible_questions = [
    (qas['question'], paragraph['context']) 
    for item in data['data'] 
    for paragraph in item['paragraphs'] 
    for qas in paragraph['qas'] if qas.get('is_impossible', True)
][:10]


In [32]:
impossible_questions

[('What category of game is Legend of Zelda: Australia Twilight?',
  'The Legend of Zelda: Twilight Princess (Japanese: ゼルダの伝説 トワイライトプリンセス, Hepburn: Zeruda no Densetsu: Towairaito Purinsesu?) is an action-adventure game developed and published by Nintendo for the GameCube and Wii home video game consoles. It is the thirteenth installment in the The Legend of Zelda series. Originally planned for release on the GameCube in November 2005, Twilight Princess was delayed by Nintendo to allow its developers to refine the game, add more content, and port it to the Wii. The Wii version was released alongside the console in North America in November 2006, and in Japan, Europe, and Australia the following month. The GameCube version was released worldwide in December 2006.[b]'),
 ('What consoles can be used to play Australia Twilight?',
  'The Legend of Zelda: Twilight Princess (Japanese: ゼルダの伝説 トワイライトプリンセス, Hepburn: Zeruda no Densetsu: Towairaito Purinsesu?) is an action-adventure game developed

In [34]:
impossible_questions_with_unique_contexts = []
unique_contexts = set()

for item in data['data']:
    for paragraph in item['paragraphs']:
        context = paragraph['context']
        if context not in unique_contexts: 
            for qas in paragraph['qas']:
                if qas.get('is_impossible', False):
                    impossible_questions_with_unique_contexts.append({
                        "question": qas['question'],
                        "context": context
                    })
                    unique_contexts.add(context)
                    break

In [36]:
len(impossible_questions_with_unique_contexts)

8853

In [38]:
import csv
output_file = "data/squad2/impossible_questions_with_unique_contexts.csv"

with open(output_file, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['question', 'context'])
    writer.writeheader()
    for item in impossible_questions_with_unique_contexts:
        writer.writerow(item)